In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

try:
    df_original = pd.read_csv('data_training.csv', parse_dates=['datetime'])
except FileNotFoundError:
    print("ERRO: Arquivo 'data_training.csv' não encontrado.")
    exit()

train_start, train_end = '2024-01-01', '2024-03-30'
test_start, test_end = '2024-04-01', '2024-06-30'

treino_df = df_original[(df_original['datetime'] >= train_start) & (df_original['datetime'] <= train_end)].copy()
teste_df = df_original[(df_original['datetime'] >= test_start) & (df_original['datetime'] <= test_end)].copy()

cols_to_drop_original = ['datetime','date','close','open','low','high','volume','average','amount_stock','id_ticker','business', 'trend']
X_train = treino_df.drop(columns=cols_to_drop_original)
y_train = treino_df['trend']

X_test = teste_df.drop(columns=cols_to_drop_original)
y_test = teste_df['trend']

modelo_original = RandomForestClassifier(n_estimators=100, random_state=42)
cv_strategy = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)

y_pred_treino_honesto = cross_val_predict(modelo_original, X_train, y_train, cv=cv_strategy)

modelo_original.fit(X_train, y_train)
y_pred_teste_honesto = modelo_original.predict(X_test)
print("Previsões para o teste geradas.")

treino_df['predicted_trend'] = y_pred_treino_honesto
treino_df['ERRORS'] = (treino_df['trend'] == treino_df['predicted_trend']).astype(int)

teste_df['predicted_trend'] = y_pred_teste_honesto
teste_df['ERRORS'] = (teste_df['trend'] == teste_df['predicted_trend']).astype(int)

df_meta = pd.concat([treino_df, teste_df], ignore_index=True)

treino_meta_df = df_meta[(df_meta['datetime'] >= train_start) & (df_meta['datetime'] <= train_end)]
teste_meta_df = df_meta[(df_meta['datetime'] >= test_start) & (df_meta['datetime'] <= test_end)]

target_meta = 'ERRORS'
features_meta_cols = X_train.columns.tolist() + ['predicted_trend']

X_train_meta = treino_meta_df[features_meta_cols]
y_train_meta = treino_meta_df[target_meta]

X_test_meta = teste_meta_df[features_meta_cols]
y_test_meta = teste_meta_df[target_meta]

meta_modelo = RandomForestClassifier(n_estimators=100, random_state=42)

y_pred_meta_cv = cross_val_predict(meta_modelo, X_train_meta, y_train_meta, cv=cv_strategy)
cm_treino_meta = confusion_matrix(y_train_meta, y_pred_meta_cv)

print("\nMatriz de Confusão do TREINO do Meta-Modelo (com CV e dados corrigidos):")
plt.figure(figsize=(7, 5))
sns.heatmap(cm_treino_meta, annot=True, fmt="d", cmap="Blues",
            xticklabels=["0", "1"],
            yticklabels=["0", "1"])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão do Treino ERRO (CV)")
plt.show()


meta_modelo.fit(X_train_meta, y_train_meta)
y_pred_meta_teste = meta_modelo.predict(X_test_meta)
cm_teste_meta = confusion_matrix(y_test_meta, y_pred_meta_teste)

print("\nMatriz de Confusão Teste ERRO:")
plt.figure(figsize=(7, 5))
sns.heatmap(cm_teste_meta, annot=True, fmt="d", cmap="Greens",
            xticklabels=["0", "1"],
            yticklabels=["0", "1"])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão do Teste ERRO")
plt.show()

print("\nRelatório de Classificação do Teste do Meta-Modelo:")
print(classification_report(y_test_meta, y_pred_meta_teste))


In [ ]:
meta_predictions_full = np.concatenate([y_pred_meta_cv, y_pred_meta_teste])
df_meta['meta_prediction'] = meta_predictions_full

df_acertos_meta = df_meta[df_meta['meta_prediction'] == 1].copy()

output_filename_meta = 'candles_previstos_como_1.csv'
df_acertos_meta.to_csv(output_filename_meta, index=False)

print(f"-> Arquivo '{output_filename_meta}' salvo com {len(df_acertos_meta)} linhas.")
print("\nPipeline concluído com sucesso!")
